In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 22 11:48:27 2019

@author: s-minhas
"""



import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import FreqDist
import operator
import matplotlib.pylab as plt
from matplotlib.pyplot import figure
from wordcloud import WordCloud
from nltk.collocations import BigramCollocationFinder 
from nltk.metrics import BigramAssocMeasures 
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
plt.rcParams.update({'font.size': 7})
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
raw_data = pd.read_csv("SMSSpamCollection.csv",  encoding='iso-8859-1') 

def split_text_to_tokens(text):
    return nltk.word_tokenize(text)

def remove_punctuation_from_tokens(tokens):
    #punctuation = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"] 
    punctuation='!?,.:;"\')(_-'
    text_without_punctuations = []
    for entity in tokens:
        newstring = ""
        for char in entity:
            if(char not in punctuation):
                  newstring+= char
        text_without_punctuations.append(newstring)
    return text_without_punctuations
  

def remove_non_alphabetic_tokens(tokens):
    alphabetic_tokens = []
    for token in tokens:
        if token.isalpha():
            alphabetic_tokens.append(token)
    return alphabetic_tokens



def remove_stopwords_from_tokens(tokens):
    stop_words = set(stopwords.words("english"))
    return [each_token for each_token in tokens if each_token not in stop_words]


def set_tokens_to_lowercase(tokens):
    
    return [each_token.lower() for each_token in tokens]

def preprocess(pstr1):
    
     s=split_text_to_tokens(pstr1)
     s=remove_non_alphabetic_tokens(s)
     s=remove_punctuation_from_tokens(s)
     s=set_tokens_to_lowercase(s)
     return s


def getsampledata(pdf, psamp):
    types = ['spam', 'ham']
    allsamples = pd.DataFrame()
    for i in types:
        data1 = pdf[pdf.Email == i]
        rows = np.random.choice(data1.index.values, psamp)
        sampled_data = pdf.loc[rows] 
        allsamples = allsamples.append(sampled_data, ignore_index=True)

    return allsamples


samp_data = getsampledata (raw_data, 10)



print ("Sample data", samp_data)

alltext = " "   

for index, row in samp_data.iterrows():
           row['Description']=' '.join(preprocess(row['Description']))
           alltext = row['Description'] + alltext
    
print ("Sample data", samp_data.head())



vectorizer = TfidfVectorizer()
samp_data_vectorised = vectorizer.fit_transform(samp_data['Description'])


tdm = pd.DataFrame(samp_data_vectorised.toarray(), columns = vectorizer.get_feature_names())
print (tdm)


from sklearn.cluster import KMeans

km = KMeans(n_clusters=2)
clusters = km.fit(tdm)
#Show counts per cluster number
print("Counts per Cluster", np.unique(clusters.labels_, return_counts=True))

#Check same number of documents returned
print("Number of documents clustered", np.unique(clusters.labels_, return_counts=True)[1].sum())

#Show number of iterations of K-means
print("number of iterations: {0}".format(clusters.n_iter_))
#add the cluster number to each input iati record
samp_data['clusterresult']=clusters.labels_
print (samp_data)


